# Assignment

WRITE IT AGAIN SUCH THAT IT ACHIEVES
- 99.4% validation accuracy
- Less than 20k Parameters
- You can use anything from above you want. 
- Less than 20 Epochs
- Have used BN, Dropout, a Fully connected layer, have used GAP. 
- To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ETC.

# Solution

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         # self.bn2  = nn.BatchNorm2d(64)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
#         # self.bn3  = nn.BatchNorm2d(128)

#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         # self.bn4  = nn.BatchNorm2d(256)

#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Conv2d(256, 512, 3)
#         # self.bn5  = nn.BatchNorm2d(512)

#         # self.conv6 = nn.Conv2d(512, 1024, 3)
#         self.conv6 = nn.Conv2d(512, 10, 1)
#         self.conv7 = nn.Conv2d(10, 10, 3)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = F.relu(self.conv7(self.conv6_1(x)))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [3]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_b1_1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5)
    self.bn1 = nn.BatchNorm2d(8)
    self.conv_b1_2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5)
    self.bn2 = nn.BatchNorm2d(16)

    self.conv_b1_drop = nn.Dropout2d(p=0.5)
    self.conv_b1_3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
    self.conv_b3_drop = nn.Dropout2d(p=0.25)
    # [<batch_size>, <channels>, <width>, <height>]
     # so, let's do the spatial averaging: reduce <width> and <height> to 1
    self.fc1 = nn.Linear(in_features=32 * 1 * 1, out_features=10)
    
    # self.pool = nn.AdaptiveAvgPool2d(1)
    # self.fc2 = nn.Linear(in_features=16, out_features=10)

  def forward(self, input_image):
    # input layer
    x = input_image

    ### mninst image handling logic

    # conv1 layer
    x = self.conv_b1_1(x)
    x = self.bn1(x)
    x = F.relu(x)
    # x = F.max_pool2d(x, kernel_size=2, stride=2) # 32, 6, 13, 13
    # print(x.shape)
    
    x =self.conv_b1_drop(x)

    # conv2 layer
    x = self.conv_b1_2(x)
    x = self.bn2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 32, 12, 5, 5
    # print(x.shape)


    # conv3 layer
    x = self.conv_b1_3(x)  # 32, 24, 6, 6
    x = self.conv_b3_drop(x)
    x = F.relu(x)
    
    # x = torch.mean(x.view(x.size(0), x.size(1), -1), dim=2)
    x = F.adaptive_avg_pool2d(x, (1, 1))
    # print(x.shape)  
    x = x.reshape(-1, 32 * 1 * 1)
    # print(x.shape)  
    
    # print(x.shape)  
    x = self.fc1(x)
    # x = self.fc2(x)
    x = F.log_softmax(x)
    return x

In [4]:
def weight_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
        nn.init.zeros_(m.bias)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
model.apply(weight_init)

summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 24, 24]             208
       BatchNorm2d-2            [-1, 8, 24, 24]              16
         Dropout2d-3            [-1, 8, 24, 24]               0
            Conv2d-4           [-1, 16, 20, 20]           3,216
       BatchNorm2d-5           [-1, 16, 20, 20]              32
            Conv2d-6             [-1, 32, 6, 6]          12,832
         Dropout2d-7             [-1, 32, 6, 6]               0
            Linear-8                   [-1, 10]             330
Total params: 16,634
Trainable params: 16,634
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.22
Params size (MB): 0.06
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)

for epoch in range(1, 12):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.5537900328636169 batch_id=468: 100%|██████████| 469/469 [00:41<00:00, 11.43it/s]



Test set: Average loss: 0.2677, Accuracy: 9253/10000 (93%)



loss=0.5410494804382324 batch_id=468: 100%|██████████| 469/469 [00:41<00:00, 11.37it/s]



Test set: Average loss: 0.1570, Accuracy: 9563/10000 (96%)



loss=0.2618153989315033 batch_id=468: 100%|██████████| 469/469 [00:41<00:00, 11.34it/s]


...


Test set: Average loss: 0.0576, Accuracy: 9837/10000 (98%)



loss=0.08561734110116959 batch_id=468: 100%|██████████| 469/469 [00:41<00:00, 11.35it/s]



Test set: Average loss: 0.0511, Accuracy: 9861/10000 (99%)



loss=0.09317105263471603 batch_id=468: 100%|██████████| 469/469 [00:41<00:00, 11.33it/s]



Test set: Average loss: 0.0515, Accuracy: 9824/10000 (98%)

